In [227]:
import googlemaps
from dotenv import load_dotenv
from googlemaps import convert
import os
import h3
import pandas as pd
from typing import List, Tuple

load_dotenv()
api_key = os.getenv('google_map_key')
gmaps = googlemaps.Client(key=api_key)

In [228]:
def plot_ruta(df):
    import folium
    # Create a map centered at the first point in the dataframe
    m = folium.Map(location=[df['lat'].iloc[0], df['lng'].iloc[0]], zoom_start=14)
    idx = 0
    # Add hexagons to the map
    for index, row in df.iterrows():
        # Get the vertices of the hexagon
        vertices = h3.h3_to_geo_boundary(row['h3'])
        
        # Convert the vertices to the format expected by folium.Polygon
        vertices = [(vertex[0], vertex[1]) for vertex in vertices]
        
        # Add the hexagon to the map
        folium.Polygon(locations=vertices, color=row['color'], fill=True, fill_color=row['color'],tooltip=idx).add_to(m)
        idx+=1
    return m

In [229]:
def get_sim_route(origin:list[float] , destination:list[float],color='blue'):
    route1 = gmaps.directions(origin,destination)
    route_poly = route1[0]['overview_polyline']['points']
    route_points = convert.decode_polyline(route_poly)
    route_df = pd.DataFrame(route_points, columns=['lat', 'lng'])

    def get_h3_index(row):
     return h3.geo_to_h3(row['lat'], row['lng'], 11)  
    
    route_df['h3'] = route_df.apply(get_h3_index, axis=1)
    route_df = route_df.drop_duplicates(subset=['h3'])

    l=[]
    hexes = route_df['h3'].values
    for i in range(0, len(hexes)-1):
        fill = h3.h3_line(hexes[i], hexes[i+1])
        fill = [item for item in fill if item != hexes[i+1]]
        l.append(fill)

    flat_list = [item for sublist in l for item in sublist]
    
    df = pd.DataFrame(columns=['lat', 'lng', 'h3'])
    for i in range(0, len(flat_list)):
        geo= h3.h3_to_geo(flat_list[i])
        temp = pd.DataFrame({'lat':geo[0],'lng':geo[1],'h3':[flat_list[i]]},index=[0])
        df = pd.concat([df, temp], ignore_index=True)
        df['color'] = color
    return df

In [230]:
places = pd.read_csv('places.csv')

In [231]:
p1 = places.sample(1).values
p2 = places.sample(1).values
df = get_sim_route(origin=p1[0][1:3],destination=p2[0][1:3])
#plot_ruta(df)

/var/folders/0v/xy628nd117v0z2tj5hql09940000gp/T/ipykernel_98011/60490761.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, temp], ignore_index=True)


In [255]:
import time
def start_ruta(TreadId:int, DeviceId:int):
    p1 = places.sample(1).values
    p2 = places.sample(1).values
    df = get_sim_route(origin=p1[0][1:3],destination=p2[0][1:3])
    deviceId = DeviceId
    host = 'localhost'
    port = '3000'
    for i in range(0, len(df),30):
        print(f"treadId:{TreadId} - {i}/{len(df)}")
        time.sleep(1)
        lat = df.iloc[i].values[0]
        lng = df.iloc[i].values[1]
        print(f"http://{host}:{port}/schedule/{deviceId}/{lat}-{lng}")

In [239]:
start_ruta()

/var/folders/0v/xy628nd117v0z2tj5hql09940000gp/T/ipykernel_98011/60490761.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, temp], ignore_index=True)


http://localhost:3000/schedule/123/14.589545802018346--90.51951156766184
http://localhost:3000/schedule/123/14.602102982572886--90.51629666588882
http://localhost:3000/schedule/123/14.611904971098141--90.5115979599997
http://localhost:3000/schedule/123/14.606613172226167--90.50174852323686
http://localhost:3000/schedule/123/14.59841221073652--90.49608090108215
http://localhost:3000/schedule/123/14.609331661147056--90.4927762122228
http://localhost:3000/schedule/123/14.603242573472096--90.48388745273297
http://localhost:3000/schedule/123/14.593233436098263--90.47856980397383
http://localhost:3000/schedule/123/14.594832254814133--90.46820617631825


In [233]:
plot_ruta(df)

In [259]:
import threading
ruta_threads = []
for i in range(5):
    t = threading.Thread(target=start_ruta, args=(i,i,))
    ruta_threads.append(t)

In [260]:
# Start the timer threads
for t in ruta_threads:
    t.start()
# Wait for all timer threads to complete
for t in ruta_threads:
    t.join()

/var/folders/0v/xy628nd117v0z2tj5hql09940000gp/T/ipykernel_98011/60490761.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, temp], ignore_index=True)
/var/folders/0v/xy628nd117v0z2tj5hql09940000gp/T/ipykernel_98011/60490761.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, temp], ignore_index=True)
/var/folders/0v/xy628nd117v0z2tj5hql09940000gp/T/ipykernel_98011/60490761.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-

treadId:4 - 0/103
treadId:1 - 0/79
treadId:2 - 0/124
treadId:3 - 0/223
http://localhost:3000/schedule/4/14.626528444254555--90.46412235814414
treadId:4 - 30/103
http://localhost:3000/schedule/1/14.619440513565067--90.51578605705268
treadId:1 - 30/79
http://localhost:3000/schedule/2/14.594372260610074--90.51576840127639
treadId:2 - 30/124
http://localhost:3000/schedule/3/14.626528444254555--90.46412235814414
treadId:3 - 30/223
http://localhost:3000/schedule/4/14.623919251357128--90.47387803032464http://localhost:3000/schedule/1/14.620671804803239--90.50951236943634
treadId:1 - 60/79
http://localhost:3000/schedule/2/14.606626113982836--90.51542847345016
treadId:2 - 60/124

treadId:4 - 60/103
http://localhost:3000/schedule/3/14.625210941028111--90.47483679381034
treadId:3 - 60/223
http://localhost:3000/schedule/1/14.607792312923934--90.51037529195186http://localhost:3000/schedule/2/14.619916293257267--90.5124747927216
treadId:2 - 90/124
http://localhost:3000/schedule/4/14.612249096567114-

[<Thread(Thread-295 (start_ruta), stopped 6307000320)>,
 <Thread(Thread-296 (start_ruta), stopped 10754224128)>,
 <Thread(Thread-297 (start_ruta), stopped 10771050496)>]